<a href="https://colab.research.google.com/github/srujan-b/Assignment-2-Choose-Your-Own-Analysis/blob/main/Validating-Synthetic-Models-as-Surgical-Training-Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Model
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,GlobalAveragePooling2D,Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
import numpy as np
from scipy.ndimage import rotate, shift,zoom
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from scipy.ndimage import rotate, zoom, affine_transform
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math
import matplotlib.pyplot as plt

In [6]:
import zipfile

zip_file_path = '/content/drive/MyDrive/ColabNotebooks/videos_imra.zip'

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('/content/drive/MyDrive/ColabNotebooks/videos_imra/')  # Change the target directory as needed
    print("Extraction successful.")
except zipfile.BadZipFile:
    print("The ZIP file appears to be corrupted.")


Extraction successful.
